In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Cours-DIC3/ML&DL Tools/project/models'

/content/drive/MyDrive/Cours-DIC3/ML&DL Tools/project/models


In [ ]:
%pip install llama_index langchain transformers sentence_transformers

In [ ]:
!nvidia-smi

In [ ]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, PromptHelper, GPTListIndex, GPTVectorStoreIndex, LLMPredictor, PromptHelper, StorageContext
import torch
from langchain import OpenAI
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms.base import LLM
from transformers import pipeline

### Extract data

In [ ]:
import extract_data

In [ ]:
extract_data.extract_data()

In [ ]:
from sentence_transformers import SentenceTransformer

class CustomLLM(LLM):
    model_name = "Sahajtomar/french_semantic"
    my_model = SentenceTransformer(model_name)

    def _call(self, prompt, stop=None):
        generated_text = self.my_model(prompt)
        return generated_text

    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"

llm_predictor = LLMPredictor(llm=CustomLLM())

In [ ]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

In [ ]:
documents = SimpleDirectoryReader('knowledge').load_data()

In [ ]:
num_output=1024
max_input_size=512
max_chunk_overlap=.20

prompt_helper = PromptHelper(max_input_size, num_output,max_chunk_overlap)

In [ ]:
index = GPTListIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor,prompt_helper=prompt_helper)

In [ ]:
response = index.as_query_engine().query("Quand l'ecole polytechnique de Thies a t-elle était créée?")

In [ ]:
response.response

In [ ]:
response2 = index.as_query_engine().query("Quel est le nom complet du directeur de l'EPT?")

In [ ]:
response2.response

In [ ]:
# Persist index to disk
index.storage_context.persist("ept_index")

In [ ]:
from llama_index import StorageContext, load_index_from_storage

# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="ept_index")

# Load index from the storage context
new_index = load_index_from_storage(storage_context)

new_query_engine = new_index.as_query_engine()
my_response = new_query_engine.query("who is this text about?")
print(my_response)